In [5]:
import pickle
import pandas as pd
import numpy as np
import sys
import time
import random
import tools
import quad
import bitlist

In [6]:
A = bitlist.read_A(1000, 50, '')
time_interval, S = bitlist.read_S(1000, 50, '')

In [55]:
A.shape

(1642, 1001)

In [56]:
from lshash import LSHash

In [84]:
def index(A, n_hash):
    column_n = A.shape[1]
    lsh = LSHash(n_hash, A.shape[0])
    for i in range(column_n):
        lsh.index(A[:,i], extra_data=i)
    return lsh
    

In [85]:
def cand_set(lsh, A, column):
    cand = set()
    ret = lsh.query(A[:,column])
    for r in ret:
        cand.add(r[0][1])
    return cand


In [88]:
lsh = index(A, 3)
cands = cand_set(lsh, A,5)
print len(cands)
sims = set(S[5].nonzero()[0])
print len(sims)
a = (cands & sims)
print len(a)

131
386
38


In [90]:
lsh = index(A, 1)
cands = cand_set(lsh, A,5)
print len(cands)
sims = set(S[5].nonzero()[0])
print len(sims)
a = (cands & sims)
print len(a)

517
386
217


In [1]:
import nearpy

In [2]:
import numpy
import scipy
import unittest

from nearpy import Engine
from nearpy.distances import CosineDistance

from nearpy.hashes import RandomBinaryProjections, RandomBinaryProjectionTree, HashPermutations, HashPermutationMapper

In [3]:
def example1():

    # Dimension of feature space
    DIM = 100

    # Number of data points (dont do too much because of exact search)
    POINTS = 10000

    print 'Creating engines'

    # We want 12 projections, 20 results at least
    rbpt = RandomBinaryProjectionTree('rbpt', 20, 20)

    # Create engine 1
    engine_rbpt = Engine(DIM, lshashes=[rbpt], distance=CosineDistance())

    # Create binary hash as child hash
    rbp = RandomBinaryProjections('rbp1', 20)

    # Create engine 2
    engine = Engine(DIM, lshashes=[rbp], distance=CosineDistance())

    # Create permutations meta-hash
    permutations = HashPermutations('permut')

    # Create binary hash as child hash
    rbp_perm = RandomBinaryProjections('rbp_perm', 20)
    rbp_conf = {'num_permutation':50,'beam_size':10,'num_neighbour':100}

    # Add rbp as child hash of permutations hash
    permutations.add_child_hash(rbp_perm, rbp_conf)

    # Create engine 3
    engine_perm = Engine(DIM, lshashes=[permutations], distance=CosineDistance())

    # Create permutations meta-hash
    permutations2 = HashPermutationMapper('permut2')

    # Create binary hash as child hash
    rbp_perm2 = RandomBinaryProjections('rbp_perm2', 12)

    # Add rbp as child hash of permutations hash
    permutations2.add_child_hash(rbp_perm2)

    # Create engine 3
    engine_perm2 = Engine(DIM, lshashes=[permutations2], distance=CosineDistance())

    print 'Indexing %d random vectors of dimension %d' % (POINTS, DIM)

    # First index some random vectors
    matrix = numpy.zeros((POINTS,DIM))
    for i in xrange(POINTS):
        v = numpy.random.randn(DIM)
        matrix[i] = v
        engine.store_vector(v)
        engine_rbpt.store_vector(v)
        engine_perm.store_vector(v)
        engine_perm2.store_vector(v)

    print 'Buckets 1 = %d' % len(engine.storage.buckets['rbp1'].keys())
    print 'Buckets 2 = %d' % len(engine_rbpt.storage.buckets['rbpt'].keys())

    print 'Building permuted index for HashPermutations'

    # Then update permuted index
    permutations.build_permuted_index()

    print 'Generate random data'

    # Get random query vector
    query = numpy.random.randn(DIM)

    # Do random query on engine 1
    print '\nNeighbour distances with RandomBinaryProjectionTree:'
    print '  -> Candidate count is %d' % engine_rbpt.candidate_count(query)
    results = engine_rbpt.neighbours(query)
    dists = [x[2] for x in results]
    print dists

    # Do random query on engine 2
    print '\nNeighbour distances with RandomBinaryProjections:'
    print '  -> Candidate count is %d' % engine.candidate_count(query)
    results = engine.neighbours(query)
    dists = [x[2] for x in results]
    print dists

    # Do random query on engine 3
    print '\nNeighbour distances with HashPermutations:'
    print '  -> Candidate count is %d' % engine_perm.candidate_count(query)
    results = engine_perm.neighbours(query)
    dists = [x[2] for x in results]
    print dists

    # Do random query on engine 4
    print '\nNeighbour distances with HashPermutations2:'
    print '  -> Candidate count is %d' % engine_perm2.candidate_count(query)
    results = engine_perm2.neighbours(query)
    dists = [x[2] for x in results]
    print dists

    # Real neighbours
    print '\nReal neighbour distances:'
    query = query.reshape((1,DIM))
    dists = CosineDistance().distance_matrix(matrix,query)
    dists = dists.reshape((-1,))
    dists = sorted(dists)
    print dists[:10]

In [4]:
example1()

Creating engines
Indexing 10000 random vectors of dimension 100
Buckets 1 = 9895
Buckets 2 = 9903
Building permuted index for HashPermutations
Generate random data

Neighbour distances with RandomBinaryProjectionTree:
  -> Candidate count is 22
[0.75279125898781385, 0.75763150529554957, 0.80132326783656938, 0.84081080722148638, 0.8456066668366814, 0.89321068449782737, 0.89448837034061901, 0.89612003980675259, 0.9286927031793234, 0.94032646714325285]

Neighbour distances with RandomBinaryProjections:
  -> Candidate count is 0
[]

Neighbour distances with HashPermutations:
  -> Candidate count is 101
[0.61847355155100758, 0.69272406855355573, 0.72854469610472705, 0.77493127650593652, 0.77593775472136406, 0.77615193483789402, 0.77676755999287983, 0.78454372702584962, 0.7929423264427109, 0.80111657615334941]

Neighbour distances with HashPermutations2:
  -> Candidate count is 230
[0.61847355155100758, 0.64072895107184324, 0.68033565105792992, 0.68124810465647556, 0.68484910535375021, 0.689

/opt/anaconda/lib/python2.7/site-packages/NearPy-0.1.2-py2.7.egg/nearpy/distances/cosine.py:54: RuntimeWarning: invalid value encountered in divide


In [50]:
DIM = A.shape[0]
permutations = HashPermutations('permut')

# Create binary hash as child hash
rbp_perm = RandomBinaryProjections('rbp_perm', 20)
rbp_conf = {'num_permutation':50,'beam_size':10,'num_neighbour':100}

# Add rbp as child hash of permutations hash
permutations.add_child_hash(rbp_perm, rbp_conf)

# Create engine 3
engine_perm = Engine(DIM, lshashes=[permutations])

for i in range(A.shape[1]):
    engine_perm.store_vector(A[:,i], data = str(i))

permutations.build_permuted_index()


In [51]:
results = engine_perm.neighbours(A[:,1])

In [52]:
rst_set = set()
for r in results:
    rst_set.add(int(r[1]))
len(rst_set)

10

In [57]:
sim_set = set(S[i].argsort()[::-1][:300])
len(sim_set)

300

In [58]:
len(sim_set & rst_set)

1

In [47]:
sim_list = S[i].argsort()[::-1][:300]

In [49]:
sim_set

{265, 351, 381, 425, 598, 627, 680, 766, 882, 893}

In [44]:
sim_list

array([381, 893, 265, 598, 882, 766, 680, 351, 627, 425])

In [46]:
len(S[i].nonzero()[0])

356